Step 1 \
Using the following method to download road map from OpenStreetMap

In [ ]:
import requests
import json
import geopandas as gpd
from shapely.geometry import LineString

# Overpass API for URL
url = "https://overpass-api.de/api/interpreter"

# Overpass QL query
query = """
[out:json];
area["name"="San Francisco"]->.sf;
(
  way["highway"](area.sf);
);
out body;
>;
out skel qt;
"""		# change name here for your own cities, more document can be found in OpenStreetMap
response = requests.get(url, params={"data": query})

if response.status_code == 200:
    data = response.json()

    # 1. save raw JSON data
    with open("san_francisco_roads.json", "w", encoding="utf-8") as f: # Change to your own cities
        json.dump(data, f, ensure_ascii=False, indent=4)
    print("raw JSON data has been saved as san_francisco_roads.json")

    # 2. analysis node data
    node_dict = {node["id"]: (node["lon"], node["lat"]) for node in data["elements"] if node["type"] == "node"}

    # 3. analysis road data and convert to GeoDataFrame
    roads = []
    for element in data["elements"]:
        if element["type"] == "way" and "nodes" in element:
            coords = [node_dict[node_id] for node_id in element["nodes"] if node_id in node_dict]
            if coords:
                roads.append(LineString(coords))

    # 4. creat GeoDataFrame
    gdf = gpd.GeoDataFrame(geometry=roads, crs="EPSG:4326")

    # 5. save road data as GeoJSON
    gdf.to_file("san_francisco_roads.geojson", driver="GeoJSON")
    print("Road data has been saved as GeoJSON：san_francisco_roads.geojson")

else:
    print(f"Fail to request, status code：{response.status_code}")


# Step 2
1. Using QGIS to convert Geojson to Esri shp(ignore it if you already have .SHP file)
2. open the shapefile in Esri ArcGIS Pro
3. Using 'Densify' tool in edit mode
4. Using 'Feature Vectices to Points' tool to convert line to point
5. Using 'Calculate Geometry Attributes' to calculate Lon and Lat
6. Export as YOUR CSV INPUT PATH.csv file

# Requirement and Output: 
What you should have after 2:
1. 'YOUR CSV INPUT PATH.csv' file from Step 2. It should store dot and its attribute 

What you should have after 2.1
1. 'YOUR_OWN_CITY_NAME_road_path_index.csv' that only stores dot's index, lon, lat

What you should have after 2.2
1. 'YOUR_OWN_CITY_NAME_panorama_id.csv', that stores PID, this is what you need for File S2


In [ ]:
# 2.1 Extract lon and lat for PID
import pandas as pd

file_path = r'YOUR CSV INPUT PATH.csv' # replace this line with your own input csv path from QGIS/ArcGIS

try:
    df = pd.read_csv(file_path, usecols=[ 'Lon', 'Lat']) #Replace the name for your own csv file, lon and lat respectively
    print(df.head())  
except ValueError as e:
    print(f"error in reading：{e}")
except FileNotFoundError:
    print(f"Cannot find file：{file_path}")
except Exception as e:
    print(f"Incidence：{e}")
    
df.reset_index(inplace=True)
print(df.head(5))

df.to_csv(r'YOUR_OWN_CITY_NAME_road_path_index.csv', index=False) #replace this line with your own csv output path, this path should be the same as 2.2

At this time, you should check your data structure of 'YOUR_OWN_CITY_NAME_road_path_index.csv', which should have 3 columns
| index | lon | lat|
|---|---|---|
|index by ArcGIS/QGIS| Longitude| Latitude|


In [ ]:
# 2.2 Search panorama ID using streetview API, RUN THE FOLLOWING CODE IN 2.2 TOGATHER
# Source code of this part is from https://github.com/ShengaoYi/Google-StreetView-Downloader
from streetview import search_panoramas
import os

path = r'YOUR_OWN_CITY_NAME_road_path_index.csv' #This path should be the smae as the output path in 2.1

fw = open(r'YOUR_OWN_CITY_NAME_panorama_id.csv', 'a', encoding='utf-8') #replace YOUR_OWN_CITY_NAME with your own city's name, for example, 'San_Francisco' will be 'SF_panorama_id.csv'. This will be the FINAL OUTPUT FILE
fw.write('pid,lat,lon,heading,pitch,roll,year,month\n')

# check_already
# This is a middle file to check if the panorama ID has been downloaded before
if not os.path.exists('YOUR_OWN_CITY_NAME_pid_got.csv'):	#replace YOUR_OWN_CITY_NAME with your own city's name, for example, 'San_Francisco' will be 'SF_pid_got.csv'
    open('YOUR_OWN_CITY_NAME_pid_got.csv', 'w', encoding='utf-8').close()	#replace YOUR_OWN_CITY_NAME with your own city's name, for example, 'San_Francisco' will be 'SF_pid_got.csv'
ALL_ID = []

# To prevent duplicate downloads
with open('YOUR_OWN_CITY_NAME_pid_got.csv', 'r', encoding='utf-8') as f:	#replace YOUR_OWN_CITY_NAME with your own city's name, for example, 'San_Francisco' will be 'SF_pid_got.csv'
  for line in f:
    ALL_ID.append(line.strip().split(".")[0])


ALL_DATA = []

n = 0

with open(path, 'r') as f:
    next(f)
    for line in f:
        n += 1
        print(n)
        line_arr = line.strip().split(',')
        xy = [float(line_arr[1]), float(line_arr[2])]
        panoids = search_panoramas(xy[1], xy[0])
        
        for panoid in panoids:
            try:
                xy = [panoid.lon, panoid.lat]
                lat = panoid.lat
                lon = panoid.lon
                pid = panoid.pano_id
                heading = panoid.heading
                pitch = panoid.pitch
                roll = panoid.roll
                date = panoid.date
                if pid in ALL_ID:
                    continue
                else:
                    ALL_ID.append(pid)
                    try:
                        year = date.split('-')[0]
                        month = date.split('-')[1]
                    except:
                        year = "None"
                        month = "None"
                    fw.write('%s,%s,%s,%s,%s,%s,%s,%s\n' % (pid, lat, lon, heading, pitch, roll, year, month))
                    fw2 = open('SF_pid_got.csv', 'a', encoding='utf-8')
                    fw2.write(pid + '\n')

            except:
                print('error')
                continue

In [ ]:
# 2.2 Search panorama ID using streetview API, RUN THE ABOVE CODE IN 2.2 TOGATHER
from streetview import search_panoramas
import os
import requests 

path = r'YOUR_OWN_CITY_NAME_road_path_index.csv'	# This path should be the same as the output path in 2.1

fw = open(r'YOUR_OWN_CITY_NAME_panorama_id.csv', 'a', encoding='utf-8')	# replace YOUR_OWN_CITY_NAME with your own city's name, for example, 'San_Francisco' will be 'SF_panorama_id.csv'

if not os.path.exists('YOUR_OWN_CITY_NAME_panorama_id.csv'):	# replace YOUR_OWN_CITY_NAME with your own city's name, for example, 'San_Francisco' will be 'SF_panorama_id.csv'
    open('YOUR_OWN_CITY_NAME_panorama_id.csv', 'w', encoding='utf-8').close()	# replace YOUR_OWN_CITY_NAME with your own city's name, for example, 'San_Francisco' will be 'SF_panorama_id.csv'

ALL_ID = []
with open('YOUR_OWN_CITY_NAME_panorama_id.csv', 'r', encoding='utf-8') as f:	# replace YOUR_OWN_CITY_NAME with your own city's name, for example, 'San_Francisco' will be 'SF_panorama_id.csv'
    for line in f:
        ALL_ID.append(line.strip().split(".")[0])

start_line = 0  # Change ID if you want to continue downloading

with open(path, 'r', encoding='utf-8') as f:
    header = next(f)  
    for line_num, line in enumerate(f, start=2):
        if line_num < start_line:
            continue

        print(f"Processing line {line_num} ...")
        line_arr = line.strip().split(',')
        xy = [float(line_arr[1]), float(line_arr[2])]  # [lon, lat]

        
        try:
            panoids = search_panoramas(xy[1], xy[0])
        except (requests.exceptions.ConnectionError, ConnectionResetError) as e:
            print(f"Network error at line {line_num}, skipping this row. Error: {e}")
            continue

        except Exception as e:
            print(f"Other error at line {line_num}, skipping. Error: {e}")
            continue

        for panoid in panoids:
            try:
                lat = panoid.lat
                lon = panoid.lon
                pid = panoid.pano_id
                heading = panoid.heading
                pitch = panoid.pitch
                roll = panoid.roll
                date = panoid.date

                if pid in ALL_ID:
                    continue
                ALL_ID.append(pid)

                try:
                    year, month = date.split('-')
                except:
                    year, month = "None", "None"

                fw.write(f"{pid},{lat},{lon},{heading},{pitch},{roll},{year},{month}\n")
                with open('SF_pid_got.csv', 'a', encoding='utf-8') as fw2:
                    fw2.write(pid + '\n')

            except Exception as e:
                print(f"Error on panoid {panoid}: {e}")
                continue

fw.close()
print("Done!")


At this time, you should check your data structure of 'YOUR_OWN_CITY_NAME_panorama_id.csv'
| pid | lat | lon | heading | pitch | roll | year | month |
|---|---|---|---|---|---|---|---|


# Now you can move to S1a
with 'YOUR_OWN_CITY_NAME_panorama_id.csv' to select pids